# Data Description

I have chosen [ERA5-land hourly data](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form) for my project. 
The objective is to find useful information about the weather in the Konkan area of Maharashtra, which receives the highest rainfall, leading to the floods. If we are able to find useful patterns in the data that could help us forecast the precipitation in the future, even by an hour, it could help the people to better prepare for the flood. 

The parameters that I have chosen are:
- `10m_v_component_of_wind` : northward component of the 10m wind, in `m/s` units, moving at a height of 10m above the surface,
- `2m_temperature` : the temperature of air at 2m above the surface, in `Kelvin` units,
- `surface_net_solar_radiation` : the amount of solar radiation that reaches a horizontal plane at the surface minus the amount reflected by the Earth's surface, in `W/m^2` units,
- `surface_pressure` : pressue of the atmosphere on the surface of land, in `Pa` units,
- `total_precipitation` : the accumulated liquid and frozen water, comprising rain and snow, that falls to the surface in `m` units.

